In [ ]:
# your code here
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
import pydeck as pdk
import pandas as pd

# bounding box for area
bbox = [-71.263053,42.289025,-71.257710,42.292009]

# query for all satellite_dishes in the area
overpass = Overpass()
query = overpassQueryBuilder(bbox=bbox, elementType='node', selector='"man_made"="satellite_dish"', out='body')
result = overpass.query(query)

# convert result data to JSON
JSON = result.toJSON()  
satellite_dishes = JSON['elements']
locations = [(satellite_dish['lon'], satellite_dish['lat']) for satellite_dish in satellite_dishes]

# create a dataframe from the locations
df = pd.DataFrame(locations, columns=['lon', 'lat'])

# create viewState and set paratmeters to lat and long near Sweden
view_state = pdk.ViewState(
    latitude=-71.265682,
    longitude=42.288675,
    zoom=9,
    min_zoom=2,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36
)

# create pydeck layer for the satellite_dishes
layer = pdk.Layer(
    'ScatterplotLayer',
    data=df,
    get_position='[lon, lat]',
    get_radius=500,
    get_fill_color=[255, 140, 0],
    pickable=True
)
# create pydeck layer for the city locations
city_layer = pdk.Layer(
    'ScatterplotLayer',
    data=None,
    get_position='[lon, lat]',
    get_radius=500,
    get_fill_color=[255, 0, 0],
    pickable=True
)

# create pydeck map
r = pdk.Deck(layers=[layer, city_layer], initial_view_state=view_state, tooltip={"text": "satellite_dish"})
r.to_html('satellite_dishes.html')




: 